# Imports

In [ ]:
import numpy as np
import cvxpy as cp # Linear optimization only

from pathlib import Path
from matplotlib import pyplot as plt
from gekko import gekko as ge # Dynamic optimization (linear, quadratic, nonlinear, and mixed integer programming)

from src.components.synthetic_house import SyntheticHouse
from src.components.synthetic_microgrid import SyntheticMicrogrid
from src.environments.simple_microgrid import SimpleMicrogrid
from src.algos.rl.a2c.d_simple_mg import Agent
from src.components.battery import Battery, BatteryParameters
from src.utils.tools import set_all_seeds, load_config

# Env tools

In [ ]:
def get_all_actions(env: SimpleMicrogrid, mode : str = 'train') -> np.ndarray:
    
    # Set mode, train, eval, test
    env.mg.change_mode(mode)

    # Create arrays to hold score, battery SOC and Action for all houses
    rewards, battery_values, action_values = [],[],[]
    
    # Same for all houses
    for house in env.mg.houses:
        reward, batt, action = solver(house)
        
        rewards.append(reward)
        battery_values.append(batt)
        action_values.append(action)
    battery_values = np.array(battery_values)
    action_values = np.array(action_values)
    rewards = np.array(rewards)

    return rewards, battery_values, action_values
    # print("Mean", scores.mean(), "\n Scores",rewards)    

In [ ]:
def loop_env(env: SimpleMicrogrid, action_values: np.ndarray, mode : str = 'train') -> np.ndarray:

    done = False
    env.mg.change_mode(mode)

    env.reset()

    # Cycle the entire episode with already computed actions by solver
    while not done:
        _,_,done,_ = env.step(action_values[:,env.mg.current_step])

    # Calculate Score with actions
    return env.mg.get_houses_metrics() # output price, emissions 

# GEKKO


## Solve single house


## To Cleanup

In [ ]:
steps = 24
exp_name = 'single_house_simple'

m = ge(remote=False, name='house')

demand = np.array([
    0.27236549, 0.28279842, 0.27286707, 0.28889338, 0.28043697,
    0.47829324, 0.93929111, 1.08548467, 0.47912301, 0.28136868,
    0.2703758 , 0.28235271, 0.28224191, 0.28233868, 0.28887496,
    0.54763641, 0.60519016, 0.60674064, 0.81195262, 0.93520451,
    0.81133533, 0.48341276, 0.33820765, 0.27257853
])

pv_gen = np.array([
    0.        , 0.        , 0.        , 0.        , 0.        ,
    0.        , 0.21511999, 0.45004407, 0.6569866 , 0.82308088,
    0.93799998, 0.99459878, 0.98935825, 0.92260421, 0.79848711,
    0.62472395, 0.41211849, 0.17388949, 0.        , 0.        ,
    0.        , 0.        , 0.        , 0.        
])

price = np.array([
    0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.12, 0.18, 0.26, 0.18, 0.12,
    0.12, 0.12, 0.12, 0.18, 0.26, 0.18, 0.12, 0.06, 0.06, 0.06, 0.06,
    0.06, 0.06
])

emission = np.array([
    0.06 , 0.06 , 0.06 , 0.06 , 0.06 , 0.06 , 0.168, 0.276, 0.42 ,
    0.276, 0.168, 0.168, 0.168, 0.168, 0.276, 0.42 , 0.276, 0.168,
    0.06 , 0.06 , 0.06 , 0.06 , 0.06 , 0.06
])

battery_eff = 1# m.Const(value=0.9)
import_rate = 0.25 #m.Const(value=0.25)

battery = m.Array(f=m.Var, dim=steps + 1, value=0.1, lb=0.1, ub=0.9)
action = m.Array(f=m.Var, dim=steps, value=0, lb=-0.8, ub=0.8)
net_energy = m.Array(f=m.Var, dim=steps, value=0)

cost = m.Array(f=m.Var, dim=steps, value=0)

equations = []

# Constraints to use all the battery energy

equations.append(battery[0] == 0.1)
equations.append(battery[steps] == 0.1)

# Define constraints for each variables

for t in range(steps):
    
    # Update battery SoC and net energy after action

    # equations.append(battery[t + 1] == battery[t] + action[t] * battery_eff)
    # equations.append(net_energy[t] == demand[t] - pv_gen[t] + action[t] * battery_eff)

    equations.append(battery[t + 1] == m.sum([battery[t], action[t] * battery_eff]))
    equations.append(net_energy[t] == m.sum([demand[t], -pv_gen[t], action[t] * battery_eff]))

    # equations.append(
    #     battery[t + 1] == m.if2(
    #         condition = action[t],
    #         x1 = m.sum([battery[t], action[t] / battery_eff ]),
    #         x2 = m.sum([battery[t], action[t] * battery_eff ]),
    #     )
    # )
    # equations.append(
    #     net_energy[t] == m.if2(
    #         condition = action[t],
    #         x1 = m.sum([demand[t], -pv_gen[t], action[t] / battery_eff ]),
    #         x2 = m.sum([demand[t], -pv_gen[t], action[t] * battery_eff ]),
    #     )
    # )

    cost[t] = m.if3(
        condition=net_energy[t], # When condition < 0, x1 is used, when condition >= 0, x2 is used
        x1= net_energy[t] * (price[t] * import_rate),
        x2= net_energy[t] * (price[t] + emission[t])
    )

# Define objective function

m.Minimize(m.sum(cost))

m.Equations(equations)

results_path = f'./results/optimizers/{exp_name}'

# Ensure the existence of results folder

Path(results_path).mkdir(parents=True, exist_ok=True)

m._path = results_path
# m.options.IMODE = 2
# m.options.SOLVER = 2
m.solve(disp=False)

m.cleanup()

# Plot results to analyze

fig = plt.figure(figsize=(10,2), num='Results', constrained_layout=True)
axs = fig.subplots(1,3, sharex=True, sharey=True)

for ax in axs:
    ax.minorticks_on()
    ax.grid(True, which='both', axis='both', alpha=0.5)

axs[0].plot(price, label='Price')
axs[0].plot(emission, label='Emission')
axs[0].legend()

axs[1].plot(demand, label='Demand')
axs[1].plot(pv_gen, label='PV generation')
axs[1].legend()

axs[2].plot([b[0] for b in battery], label='Battery')
axs[2].plot([a[0] for a in action], label='Action')
axs[2].legend()

plt.show()

# print(action)
print(net_energy)
print(f'Objective: {m.options.objfcnval:.2f}')

In [ ]:
# Plot results to analyze

fig = plt.figure(figsize=(10,2), num='Results', constrained_layout=True)
axs = fig.subplots(1,3, sharex=True, sharey=True)

for ax in axs:
    ax.minorticks_on()
    ax.grid(True, which='both', axis='both', alpha=0.5)

axs[0].plot(price, label='Price')
axs[0].plot(emission, label='Price')

axs[1].plot(demand, label='Demand')
axs[1].plot(pv_gen, label='PV generation')

axs[2].plot([b[0] for b in battery], label='Battery')
axs[2].plot([a[0] for a in action], label='Action')

print(action)
print(net_energy)
print(f'Objective: {m.options.objfcnval:.2f}')

## Current house

In [ ]:
def solver_house(h: SyntheticHouse, steps: int = 24, reset: bool = True, plot: bool = False, exp_name: str = 'single_house'):

    m = ge(remote=False, name='house')

    battery = m.Array(f=m.Var, dim=steps + 1, value=h.battery.soc_min, lb=h.battery.soc_min, ub=h.battery.soc_max)
    action = m.Array(f=m.Var, dim=steps, value=0, lb=-h.battery.p_discharge_max, ub=h.battery.p_charge_max)
    net_energy = m.Array(f=m.Var, dim=steps, value=0)
    
    cost = m.Array(f=m.Var, dim=steps, value=0)
    
    equations = []

    # Constraints to use all the battery energy

    equations.append(battery[0] == h.battery.soc_min)
    equations.append(battery[steps] == h.battery.soc_min)

    # Define constraints for each variables

    for t in range(steps):
        
        # Update battery SoC and net energy after action

        # equations.append(battery[t + 1] == m.sum([battery[t], action[t] * h.battery.efficiency]))
        # equations.append(net_energy[t] == m.sum([h.demand[t], -h.pv_gen[t], action[t] * h.battery.efficiency]))

        equations.append(
            battery[t + 1] == m.if2(
                condition = action[t],
                x1 = m.sum([battery[t], action[t] / h.battery.efficiency ]),
                x2 = m.sum([battery[t], action[t] * h.battery.efficiency ]),
            )
        )
        equations.append(
            net_energy[t] == m.if2(
                condition = action[t],
                x1 = m.sum([h.demand[t], -h.pv_gen[t], action[t] / h.battery.efficiency ]),
                x2 = m.sum([h.demand[t], -h.pv_gen[t], action[t] * h.battery.efficiency ]),
            )
        )

        cost[t] = m.if3(
            condition=net_energy[t], # When condition < 0, x1 is used, when condition >= 0, x2 is used
            x1= net_energy[t] * (h.l3_export_rate[t] * h.l3_import_fraction),
            x2= net_energy[t] * (h.l3_export_rate[t] + h.l3_emission[t])
        )

    # Define objective function

    m.Minimize(m.sum(cost))

    m.Equations(equations)
    
    results_path = f'./results/optimizers/{exp_name}'

    # Ensure the existence of results folder

    Path(results_path).mkdir(parents=True, exist_ok=True)

    m._path = results_path
    # m.options.IMODE = 2
    # m.options.SOLVER = 2
    m.solve(disp=False)
    
    if plot:

    # Plot results to analyze

        fig = plt.figure(figsize=(10,2), num='Results', constrained_layout=True)
        axs = fig.subplots(1,3, sharex=True, sharey=True)

        for ax in axs:
            ax.minorticks_on()
            ax.grid(True, which='both', axis='both', alpha=0.5)

        axs[0].plot(h.l3_export_rate, label='Price')
        axs[0].plot(h.l3_emission, label='Emission')
        axs[0].legend()

        axs[1].plot(h.demand, label='Demand')
        axs[1].plot(h.pv_gen, label='PV generation')
        axs[1].legend()

        axs[2].plot([b[0] for b in battery], label='Battery')
        axs[2].plot([a[0] for a in action], label='Action')
        axs[2].legend()

        plt.show()

    # Print results and cleanup

    print(f'Objective: {m.options.objfcnval:.2f}')

    if reset:
        m.cleanup()

    return m.options.objfcnval


# Test the solver

set_all_seeds(0)

model = "d_a2c_mg"

config = load_config(model)
config = config['train']

my_env = SimpleMicrogrid(config=config['env'])

# Compute optimal results

results = []

results.append(solver_house(h=my_env.mg.houses[0], steps=24))
results.append(solver_house(h=my_env.mg.houses[1], steps=24))
results.append(solver_house(h=my_env.mg.houses[2], steps=24))
results.append(solver_house(h=my_env.mg.houses[3], steps=24))
results.append(solver_house(h=my_env.mg.houses[4], steps=24))
results.append(solver_house(h=my_env.mg.houses[5], steps=24))

print(f'Average: {np.mean(results)}')

## Microgrid


### Minimize price metric

In [ ]:
def solver_microgrid(mg: SyntheticMicrogrid, steps: int = 24, reset: bool = True, plot: bool = False):

    exp_name = 'microgrid'

    m = ge(remote=False, name='microgrid')

    n_houses = len(mg.houses)

    # Variables

    # Initializing this way allows to put different ub and lb for each house
    battery = [m.Array(f=m.Var, dim=(steps + 1), value=h.battery.soc_min, lb=h.battery.soc_min, ub=h.battery.soc_max) for h in mg.houses] 
    action = [m.Array(f=m.Var, dim=(steps), value=0, lb=-h.battery.p_discharge_max, ub=h.battery.p_charge_max) for h in mg.houses]

    net_energy = m.Array(f=m.Var, dim=(n_houses,steps), value=0)
    l1_import = m.Array(f=m.Var, dim=(n_houses,steps), value=0, lb=0)
    l1_export = m.Array(f=m.Var, dim=(n_houses,steps), value=0, lb=0)

    # Parameters

    surplus_energy = m.Array(f=m.Param, dim=(n_houses,steps), value=0)
    shortage_energy = m.Array(f=m.Param, dim=(n_houses,steps), value=0)
    offer = m.Array(f=m.Param, dim=(steps), value=0)
    demand = m.Array(f=m.Param, dim=(steps), value=0)
    l1_import_rate = m.Array(f=m.Param, dim=(steps), value=0)
    l1_export_rate = m.Array(f=m.Param, dim=(steps), value=0)
    step_transacted_energy = m.Array(f=m.Param, dim=(steps), value=0)
    real_net_energy = m.Array(f=m.Param, dim=(n_houses,steps), value=0)
    
    # Cost function initialization

    cost = [m.Array(f=m.Var, dim=steps, value=0) for _ in mg.houses]
    
    equations = []

    # Constraints to use all the battery energy in the cycle

    for house_ix, house in enumerate(mg.houses):
        equations.append(battery[house_ix][0] == house.battery.soc_min)
        equations.append(battery[house_ix][steps] == house.battery.soc_min)

    # Define constraints for each variable

    for t in range(steps):
        
        for house_ix, house in enumerate(mg.houses):

            # Update battery SoC and net energy after action

            equations.append(
                battery[house_ix][t + 1] == m.if2(
                    condition = action[house_ix][t],
                    x1 = m.sum([battery[house_ix][t], action[house_ix][t] / house.battery.efficiency ]),
                    x2 = m.sum([battery[house_ix][t], action[house_ix][t] * house.battery.efficiency ]),
                )
            )
            equations.append(
                net_energy[house_ix][t] == m.if2(
                    condition = action[house_ix][t],
                    x1 = m.sum([house.demand[t], -house.pv_gen[t], action[house_ix][t] / house.battery.efficiency ]),
                    x2 = m.sum([house.demand[t], -house.pv_gen[t], action[house_ix][t] * house.battery.efficiency ]),
                )
            )

            # Define surplus and shortage energy

            surplus_energy[house_ix][t] =  m.max2(-net_energy[house_ix][t],0)
            shortage_energy[house_ix][t] = m.max2(net_energy[house_ix][t],0)

        # Define offer and demand

        offer[t] = m.sum([surplus_energy[house_ix][t] for house_ix in range(n_houses)])
        demand[t] = m.sum([shortage_energy[house_ix][t] for house_ix in range(n_houses)])

        # Define import and export rates

        baseline = mg.price[t] * mg.current_profile['import_fraction']
        l1_import_rate[t] = m.sum([baseline + mg.current_profile['l1_alpha'] * demand[t], -mg.current_profile['l1_beta'] * offer[t]])
        l1_export_rate[t] = m.sum([l1_import_rate[t], -mg.current_profile['l1_fee']])

        # Transaction contraints

        step_transacted_energy[t] = m.if2(condition=(m.sum([offer[t], demand[t]])), x1=offer[t], x2=demand[t])

        equations.append(step_transacted_energy[t] == m.sum([l1_export[house_ix][t] for house_ix in range(n_houses)]))
        equations.append(step_transacted_energy[t] == m.sum([l1_import[house_ix][t] for house_ix in range(n_houses)]))

        for house_ix, house in enumerate(mg.houses):

            equations.append(m.sum([l1_export[house_ix][t], l1_import[house_ix][t]]) > 0)
            equations.append((l1_export[house_ix][t] * l1_import[house_ix][t]) == 0)
            equations.append(l1_export[house_ix][t] <= surplus_energy[house_ix][t])
            equations.append(l1_export[house_ix][t] >= 0)
            equations.append(l1_import[house_ix][t] <= shortage_energy[house_ix][t])
            equations.append(l1_import[house_ix][t] >= 0)

            real_net_energy[house_ix][t] = m.sum([net_energy[house_ix][t],l1_export[house_ix][t], -l1_import[house_ix][t]])

            cost[house_ix][t] = m.if3(
                condition=real_net_energy[house_ix][t],
                x1= real_net_energy[house_ix][t] * (mg.price[t] * mg.current_profile['import_fraction']) - l1_export[house_ix][t] * l1_export_rate[t] + l1_import[house_ix][t] * l1_import_rate[t],
                x2= real_net_energy[house_ix][t] * (mg.price[t]) + l1_import[house_ix][t] * l1_import_rate[t],
            )

    # Define objective function

    objs_cost = np.sum(cost, axis=1)

    for house_ix, o_c in enumerate(objs_cost):

        m.Minimize(o_c - mg.houses[house_ix].compute_metrics_no_batt()[0])

    m.Equations(equations)
    
    results_path = f'./results/optimizers/{exp_name}'

    # Ensure the existence of results folder

    Path(results_path).mkdir(parents=True, exist_ok=True)

    m._path = results_path
    # m.options.IMODE = 2
    # m.options.SOLVER = 0
    
    m.solve(disp=False)

    results = [c[0].value[0] for c in cost]

    # Convert results to numpy arrays

    battery = np.array([[b[0] for b in bh] for bh in battery])
    net_energy = np.array([[n[0] for n in nh] for nh in net_energy])
    action = np.array([[a[0] for a in ah] for ah in action])
    l1_export = np.array([[e[0] for e in eh] for eh in l1_export])
    l1_import = np.array([[i[0] for i in ih] for ih in l1_import])
    offer = np.array([o[0] for o in offer])
    demand = np.array([d[0] for d in demand])
    l1_export_rate = np.array([e[0] for e in l1_export_rate])
    l1_import_rate = np.array([i[0] for i in l1_import_rate])
    surplus_energy = np.array([[s[0] for s in sp] for sp in surplus_energy])
    shortage_energy = np.array([[s[0] for s in sh] for sh in shortage_energy])
    real_net_energy = np.array([[n[0] for n in net] for net in real_net_energy])

    # Plot results to analyze the optimization

    if plot:

        for h_ix, h in enumerate(mg.houses):

            fig = plt.figure(figsize=(12,2), num='Results', constrained_layout=True)
            axs = fig.subplots(1,4, sharex=True, sharey=False)

            for ax in axs:
                ax.minorticks_on()
                ax.grid(True, which='both', axis='both', alpha=0.5)

            axs[0].plot(h.l3_export_rate, label='L3 Export')
            axs[0].plot(h.l3_emission, label='L4 Emission')
            axs[0].plot(l1_export_rate, label='L1 Export')
            axs[0].plot(l1_import_rate, label='L1 Import')
            axs[0].legend()

            axs[1].plot(net_energy[h_ix], label='Net')
            axs[1].plot(real_net_energy[h_ix], label='Real Net')
            axs[1].plot(mg.houses[h_ix].remaining_energy, label='Remaining')
            axs[1].legend()

            axs[2].plot(battery[h_ix], label='Battery')
            axs[2].plot(action[h_ix], label='Action')
            axs[2].legend()

            axs[3].plot(l1_import[h_ix], label='L1 import')
            axs[3].plot(l1_export[h_ix], label='L1 export')
            axs[3].legend()

            plt.show()

    plt.plot(offer, label='Offer')
    plt.plot(demand, label='Demand')
    plt.plot(np.sum(l1_import, axis=0), label='Total L1 import')
    plt.plot(np.sum(l1_export, axis=0), label='Total L1 export')
    plt.grid(True, which='both', axis='both', alpha=0.5)
    plt.legend()

    plt.show()

    # Print results and cleanup

    [print(f'House {i} cost: {r}') for i, r in enumerate(results)]
    print(f'Average: {np.mean(results)}')
    print(f'Objective: {m.options.objfcnval:.2f}')

    if reset:
        m.cleanup()

    return results, action

# Test the solver

set_all_seeds(0)

model = "d_a2c_mg"

config = load_config(model)
config = config['train']

my_env = SimpleMicrogrid(config=config['env'])

profile_costs = []
profile_actions = []

for i in config['env']['grid']['profiles']:
    
    results, action = solver_microgrid(mg=my_env.mg, steps=config['env']['rollout_steps'], plot=True, reset=True)
    profile_costs.append(results)
    profile_actions.append(action)

    print(f'{i}: {np.array(results).mean()}')

    my_env.mg.change_grid_profile()

### Minimize price metric and emissions

In [ ]:
def solver_microgrid(mg: SyntheticMicrogrid, steps: int = 24, reset: bool = True, plot: bool = False):

    exp_name = 'microgrid'

    m = ge(remote=False, name='microgrid')

    n_houses = len(mg.houses)

    # Variables

    # Initializing this way allows to put different ub and lb for each house
    battery = [m.Array(f=m.Var, dim=(steps + 1), value=h.battery.soc_min, lb=h.battery.soc_min, ub=h.battery.soc_max) for h in mg.houses] 
    action = [m.Array(f=m.Var, dim=(steps), value=0, lb=-h.battery.p_discharge_max, ub=h.battery.p_charge_max) for h in mg.houses]
    net_energy = m.Array(f=m.Var, dim=(n_houses,steps), value=0)
    l1_import = m.Array(f=m.Var, dim=(n_houses,steps), value=0, lb=0)
    l1_export = m.Array(f=m.Var, dim=(n_houses,steps), value=0, lb=0)

    # Parameters

    surplus_energy = m.Array(f=m.Param, dim=(n_houses,steps), value=0)
    shortage_energy = m.Array(f=m.Param, dim=(n_houses,steps), value=0)
    offer = m.Array(f=m.Param, dim=(steps), value=0)
    demand = m.Array(f=m.Param, dim=(steps), value=0)
    l1_import_rate = m.Array(f=m.Param, dim=(steps), value=0)
    l1_export_rate = m.Array(f=m.Param, dim=(steps), value=0)
    step_transacted_energy = m.Array(f=m.Param, dim=(steps), value=0)
    real_net_energy = m.Array(f=m.Param, dim=(n_houses,steps), value=0)
    
    # Cost function initialization

    cost = [m.Array(f=m.Var, dim=steps, value=0) for _ in mg.houses]
    emission = [m.Array(f=m.Var, dim=steps, value=0) for _ in mg.houses]
    
    equations = []

    # Constraints to use all the battery energy in the cycle

    for house_ix, house in enumerate(mg.houses):
        equations.append(battery[house_ix][0] == house.battery.soc_min)
        equations.append(battery[house_ix][steps] == house.battery.soc_min)

    # Define constraints for each variable

    for t in range(steps):
        
        for house_ix, house in enumerate(mg.houses):

            # Update battery SoC and net energy after action

            equations.append(
                battery[house_ix][t + 1] == m.if2(
                    condition = action[house_ix][t],
                    x1 = m.sum([battery[house_ix][t], action[house_ix][t] / house.battery.efficiency ]),
                    x2 = m.sum([battery[house_ix][t], action[house_ix][t] * house.battery.efficiency ]),
                )
            )
            equations.append(
                net_energy[house_ix][t] == m.if2(
                    condition = action[house_ix][t],
                    x1 = m.sum([house.demand[t], -house.pv_gen[t], action[house_ix][t] / house.battery.efficiency ]),
                    x2 = m.sum([house.demand[t], -house.pv_gen[t], action[house_ix][t] * house.battery.efficiency ]),
                )
            )

            # Define surplus and shortage energy

            surplus_energy[house_ix][t] =  m.max2(-net_energy[house_ix][t],0)
            shortage_energy[house_ix][t] = m.max2(net_energy[house_ix][t],0)

        # Define offer and demand

        offer[t] = m.sum([surplus_energy[house_ix][t] for house_ix in range(n_houses)])
        demand[t] = m.sum([shortage_energy[house_ix][t] for house_ix in range(n_houses)])

        # Define import and export rates

        baseline = mg.price[t] * mg.current_profile['import_fraction']
        l1_import_rate[t] = m.sum([baseline + mg.current_profile['l1_alpha'] * demand[t], -mg.current_profile['l1_beta'] * offer[t]])
        l1_export_rate[t] = m.sum([l1_import_rate[t], -mg.current_profile['l1_fee']])

        # Transaction contraints

        step_transacted_energy[t] = m.if2(condition=(m.sum([offer[t], demand[t]])), x1=offer[t], x2=demand[t])

        equations.append(step_transacted_energy[t] == m.sum([l1_export[house_ix][t] for house_ix in range(n_houses)]))
        equations.append(step_transacted_energy[t] == m.sum([l1_import[house_ix][t] for house_ix in range(n_houses)]))

        for house_ix, house in enumerate(mg.houses):

            equations.append(m.sum([l1_export[house_ix][t], l1_import[house_ix][t]]) > 0)
            equations.append((l1_export[house_ix][t] * l1_import[house_ix][t]) == 0)
            equations.append(l1_export[house_ix][t] <= surplus_energy[house_ix][t])
            equations.append(l1_export[house_ix][t] >= 0)
            equations.append(l1_import[house_ix][t] <= shortage_energy[house_ix][t])
            equations.append(l1_import[house_ix][t] >= 0)

            real_net_energy[house_ix][t] = m.sum([net_energy[house_ix][t],l1_export[house_ix][t], -l1_import[house_ix][t]])

            cost[house_ix][t] = m.if3(
                condition=real_net_energy[house_ix][t],
                x1= real_net_energy[house_ix][t] * (mg.price[t] * mg.current_profile['import_fraction']) - l1_export[house_ix][t] * l1_export_rate[t] + l1_import[house_ix][t] * l1_import_rate[t],
                x2= real_net_energy[house_ix][t] * (mg.price[t]) + l1_import[house_ix][t] * l1_import_rate[t],
            )

            emission[house_ix][t] = m.if3(
                condition=real_net_energy[house_ix][t],
                x1= 0,
                x2= real_net_energy[house_ix][t] * (mg.emission[t]),
            )

    # Define objective function

    objs_cost = np.sum(cost, axis=1)
    objs_emission = np.sum(emission, axis=1)

    for house_ix, (o_c, o_e) in enumerate(zip(objs_cost, objs_emission)):

        m.Minimize(o_c - mg.houses[house_ix].compute_metrics_no_batt()[0] + o_e - mg.houses[house_ix].compute_metrics_no_batt()[1])

    # m.Minimize(obj)
    
    m.Equations(equations)
    
    results_path = f'./results/optimizers/{exp_name}'

    # Ensure the existence of results folder

    Path(results_path).mkdir(parents=True, exist_ok=True)

    m._path = results_path
    # m.options.IMODE = 2
    # m.options.SOLVER = 0
    
    m.solve(disp=False)

    # results = [np.sum([c[0] for c in cost[h_ix]]) for h_ix in range(n_houses)]
    results = [c[0].value for c in cost]

    # Convert results to numpy arrays

    battery = np.array([[b[0] for b in bh] for bh in battery])
    net_energy = np.array([[n[0] for n in nh] for nh in net_energy])
    action = np.array([[a[0] for a in ah] for ah in action])
    l1_export = np.array([[e[0] for e in eh] for eh in l1_export])
    l1_import = np.array([[i[0] for i in ih] for ih in l1_import])
    offer = np.array([o[0] for o in offer])
    demand = np.array([d[0] for d in demand])
    l1_export_rate = np.array([e[0] for e in l1_export_rate])
    l1_import_rate = np.array([i[0] for i in l1_import_rate])
    surplus_energy = np.array([[s[0] for s in sp] for sp in surplus_energy])
    shortage_energy = np.array([[s[0] for s in sh] for sh in shortage_energy])
    real_net_energy = np.array([[n[0] for n in net] for net in real_net_energy])

    # Plot results to analyze the optimization

    if plot:

        for h_ix, h in enumerate(mg.houses):

            fig = plt.figure(figsize=(10,5), num='Results', constrained_layout=True)
            axs = fig.subplots(1,4, sharex=True, sharey=False)

            for ax in axs:
                ax.minorticks_on()
                ax.grid(True, which='both', axis='both', alpha=0.5)

            axs[0].plot(h.l3_export_rate, label='L3 Export')
            axs[0].plot(h.l3_emission, label='L4 Emission')
            axs[0].plot(l1_export_rate, label='L1 Export')
            axs[0].plot(l1_import_rate, label='L1 Import')
            axs[0].legend()

            axs[1].plot(net_energy[h_ix], label='Net')
            axs[1].plot(real_net_energy[h_ix], label='Real Net')
            axs[1].legend()

            axs[2].plot(battery[h_ix], label='Battery')
            axs[2].plot(action[h_ix], label='Action')
            axs[2].legend()

            axs[3].plot(l1_import[h_ix], label='L1 import')
            axs[3].plot(l1_export[h_ix], label='L1 export')
            axs[3].legend()

            plt.show()

    plt.plot(offer, label='Offer')
    plt.plot(demand, label='Demand')
    plt.plot(np.sum(l1_import, axis=0), label='Total L1 import')
    plt.plot(np.sum(l1_export, axis=0), label='Total L1 export')
    plt.legend()

    plt.show()

    # Print results and cleanup

    [print(f'House {i} cost: {r}') for i, r in enumerate(results)]
    print(f'Average: {np.mean(results)}')
    print(f'Objective: {m.options.objfcnval:.2f}')

    if reset:
        m.cleanup()

    return results, action

# Test the solver

set_all_seeds(0)

model = "d_a2c_mg"

config = load_config(model)
config = config['train']

my_env = SimpleMicrogrid(config=config['env'])

results, action = solver_microgrid(mg=my_env.mg, steps=config['env']['rollout_steps'], plot=True, reset=True)


# Debug

In [ ]:
def loop_env(env: SimpleMicrogrid, action_values: np.ndarray, mode : str = 'train') -> np.ndarray:

    done = False
    env.mg.change_mode(mode)

    env.reset()

    # Cycle the entire episode with already computed actions by solver
    while not done:
        _,_,done,_ = env.step(action_values[:,env.mg.current_step])

    # Calculate Score with actions
    return env.mg.get_houses_metrics() # output price, emissions 

In [ ]:
# Check the metrics with optimal actions

set_all_seeds(0)

model = "d_a2c_mg"

config = load_config(model)
config = config['train']

my_env = SimpleMicrogrid(config=config['env'])

for action in profile_actions:

    m_price, m_emissions = loop_env(my_env, action)
    my_env.mg.change_grid_profile()

    print(f'Average price: {np.mean(m_price)} prices, {m_price}')
    print(f'Average emission: {np.mean(m_emissions)} emissions: {m_emissions}')

# CVXPY


## Functions


In [ ]:
# THIS CASE SEEMS TO NOT BE SOLVABLE WITH CVXPY (NON CONVEX PROBLEM)

def solver_microgrid(mg: SyntheticMicrogrid, steps: int = 24):

    n_houses = len(mg.houses)

    battery = cp.Variable(shape=(steps + 1, n_houses), name='battery')
    action = cp.Variable(shape=(steps, n_houses), name='action')
    net_energy = cp.Variable(shape=(steps, n_houses), name='net_energy')
    # true_net_energy = cp.Variable(shape=(steps, n_houses))
    # l1_import = cp.Variable(shape=(steps, n_houses))
    # l1_export = cp.Variable(shape=(steps, n_houses))
    # l1_import_rate = cp.Variable(shape=(steps))
    # l1_export_rate = cp.Variable(shape=(steps))
    
    constraints = []

    # Define constraints for each variables

    for house_ix, house in enumerate(mg.houses):

        # Initial SoC

        constraints.append(battery[0, house_ix] == house.battery.soc_min)

        # Constraints for each time step

        for t in range(steps):

            # Soc within the limits

            constraints.append(battery[t + 1, house_ix] >= house.battery.soc_min)
            constraints.append(battery[t + 1, house_ix] <= house.battery.soc_max)

            # Action within the limits

            constraints.append(action[t, house_ix] <= house.battery.p_charge_max)
            constraints.append(action[t, house_ix] >= -house.battery.p_discharge_max)

            # Update battery SoC and net energy after action

            constraints.append(battery[t + 1, house_ix] == battery[t, house_ix] + action[t, house_ix] * house.battery.efficiency)
            constraints.append(net_energy[t, house_ix] == house.demand[t] - house.pv_gen[t] * action[t, house_ix] * house.battery.efficiency)
            
    # Compute transactions

    surplus = cp.maximum(-net_energy, 0)
    shortage = cp.maximum(net_energy, 0)
    offer = cp.sum(surplus, axis=1)
    demand = cp.sum(shortage, axis=1)

    # Transactions and objective function definition

    obj = 0

    for t in range(steps):

        step_offer = offer[t]
        step_demand = demand[t]

        baseline = float(mg.price[t] * mg.current_profile['import_fraction'])
        # constraints.append(l1_import_rate[t] == baseline + mg.current_profile['l1_alpha'] * step_demand - mg.current_profile['l1_beta'] * step_offer)
        # constraints.append(l1_export_rate[t] == l1_import_rate - mg.current_profile['l1_fee'])
        l1_import_rate = baseline + mg.current_profile['l1_alpha'] * step_demand - mg.current_profile['l1_beta'] * step_offer
        l1_export_rate = l1_import_rate - mg.current_profile['l1_fee']
        
        for house_ix, house in enumerate(mg.houses):

            # Compute the energy transacted and the contribution of each house

            step_transacted_energy = cp.abs(step_demand - step_offer)
            # step_transacted_energy = cp.minimum(step_offer, step_demand)
            step_surplus_distribution = step_transacted_energy * (surplus[t, house_ix] / step_offer)
            step_shortage_distribution = step_transacted_energy * (shortage[t, house_ix] / step_demand)

            # constraints.append(l1_export[t,house_ix] == step_surplus_distribution)
            # constraints.append(l1_import[t,house_ix] == step_shortage_distribution)
            # constraints.append(true_net_energy[t,house_ix] == net_energy[t,house_ix] + (step_surplus_distribution - step_shortage_distribution))

            l1_export = step_surplus_distribution
            l1_import = step_shortage_distribution
            true_net_energy = net_energy[t,house_ix] + (step_surplus_distribution - step_shortage_distribution)

            # Compute the objective function

            obj += cp.maximum((true_net_energy + l1_export - l1_import) * (mg.price[t] + mg.emission[t]) + l1_import * l1_import_rate, 0) 
            obj += cp.maximum(-(true_net_energy + l1_export - l1_import) * mg.price[t] * mg.current_profile['import_fraction'] + l1_export * l1_export_rate, 0) 

            # obj += cp.maximum(net_energy[t,house_ix] * (mg.price[t] + mg.emission[t]), 0) 
            # obj += cp.maximum(-net_energy[t,house_ix] * mg.price[t] * mg.current_profile['import_fraction'], 0) 

    # Solve the problem

    objective = cp.Minimize(obj)
    prob = cp.Problem(objective, constraints)
    res = prob.solve(verbose=True)

    return res, battery.value, action.value


In [ ]:
model = "d_a2c_mg"

config = load_config(model)
config = config['train']

my_env = SimpleMicrogrid(config=config['env'])

solver_microgrid(mg=my_env.mg, steps=2)

In [ ]:
def solver(house: SyntheticHouse, n: int = 24):

    battery = cp.Variable(n+1)
    action = cp.Variable(n)
    consumption = cp.Variable(n)

    constraints = []

    # Battery
        # Starts in 0.1
    constraints.append(battery[0] == house.battery.soc_min)
        # Max and min batteries
    for i in range(n+1):
        constraints.append(battery[i] <= house.battery.soc_max)
        constraints.append(battery[i] >= house.battery.soc_min)


    # Action / Batteryn't

    for i in range(n):
        constraints.append(action[i] <= 1)
        constraints.append(action[i] >= -1)


    # Transition
    obj = 0

    for i in range(n):
        
        constraints.append(action[i] <= house.battery.p_charge_max)
        constraints.append(action[i] <= house.battery.p_discharge_max)
        # Update battery
        constraints.append(battery[i+1] == battery[i] + action[i] * house.battery.efficiency)
        # Update net 
        constraints.append(consumption[i] == house.demand[i]-house.pv_gen[i] + action[i] * house.battery.efficiency)


        obj += cp.maximum(consumption[i] * (house.price[i] + house.emission[i]),0) 
        obj += cp.maximum(-consumption[i] * house.price[i] * house.grid_sell_rate,0)  


    objective = cp.Minimize(obj)
    prob = cp.Problem(objective, constraints)
    res = prob.solve()

    return res, battery.value, action.value

## Run this


In [ ]:
set_all_seeds(0)
# create environment,m save array of houses
config = load_config("zero_mg")
env = SimpleMicrogrid(config=config['env'])

# Train
mode = 'train'
rewards, battery_values, action_values = get_all_actions(env, mode)
train_metrics = loop_env(env, action_values, mode)
print(rewards[0].mean())
print('train ', train_metrics)

# Eval
mode = 'eval'
rewards, battery_values, action_values = get_all_actions(env, mode)
print(rewards[0].mean())
eval_metrics = loop_env(env, action_values, mode)

# Test
mode = 'test'
rewards, battery_values, action_values = get_all_actions(env, mode)
print(rewards[0].mean())
test_metrics = loop_env(env, action_values, mode)


In [ ]:
np.array(train_metrics).mean(axis=1)

In [ ]:
train_metrics, eval_metrics, test_metrics

In [ ]:
set_all_seeds(0)

model = "d_a2c_mg"
config = load_config(model)
config = config['train']
my_env = SimpleMicrogrid(config=config['env'])

agent = Agent(env=my_env, config = config)
results_ag = agent.train()
results_ag['test'] = agent.test()
agent.wdb_logger.finish()

# House


### Set env vars


In [ ]:
set_all_seeds(0)

# Same env obs as test simple env

config = load_config("c_a2c")
config = config['train']

mg = SyntheticHouse(config=config['env'])
demand = mg.demand
pv = mg.pv_gen
price_s = mg.price
emission = mg.emission
price_b = price_s / 4

In [ ]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
    
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

### Adv battery calc (not working)


In [ ]:
set_all_seeds(0)
battery = cp.Variable(n+1)
action = cp.Variable(n)
consumption = cp.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't
# for i in range(n):
#     constraints.append(action[i] <= 0.9)
#     constraints.append(action[i] >= -0.9)

# Transition
obj = 0

for i in range(n):
    # Max and min battery charge 
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery SOC
    # self.soc = self.soc + (p_charge * self.efficiency - p_discharge / self.efficiency) / self.capacity

    constraints.append(battery[i+1] == battery[i] + ((action[i] * battery_params["efficiency"])/battery_params["efficiency"])/battery_params["capacity"] )
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cp.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cp.maximum(-consumption[i] * price_b[i],0)  


objective = cp.Minimize(obj)
prob = cp.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value, consumption.value, price_s

### Real dataset


In [ ]:
n = 24

battery = cp.Variable(n+1)
action = cp.Variable(n)
consumption = cp.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery
    constraints.append(battery[i+1] == battery[i] + action[i] * battery_params["efficiency"])
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cp.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cp.maximum(-consumption[i] * price_b[i],0)  


objective = cp.Minimize(obj)
prob = cp.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value